In [ ]:
%matplotlib inline
import matplotlib.pyplot as ptl
from PIL import Image
from pathlib import Path
import random
import shutil

def resize_images(source_dir, target_dir, test_samples):
  
  source_dir = Path(source_dir)
  target_dir = Path(target_dir)

  try:
    target_dir.mkdir()
    train_path = Path(target_dir/"train")
    train_path.mkdir()
    
    test_path = Path(target_dir/"test")
    test_path.mkdir()
  except FileExistsError:
    print("Target Directory already exists.")


  for dir in source_dir.iterdir():
    print(f"Directory name: {dir.name}")
    print("Images in directory")
    x = 0
    for image in dir.iterdir():
      print(f"Resizing: {image.name}")
      img = Image.open(image)
      img = img.convert("RGB")
      
#      plt.imshow(img)
#      plt.axis('on')
#      plt.title(image)
#      plt.show()

      img = img.resize((32,32))
#      plt.imshow(img)
#      plt.axis('on')
#      plt.title(image)
#     plt.show()
      img.save(train_path/f"{dir.name}_{x}.jpg")
      x = x+1
    samples = random.sample(range(0, x), test_samples)
    
    for sample in samples:
      print(f"New Test Sample: {dir.name}_{sample}.jpg")
      shutil.move(target_dir/f"train/{dir.name}_{sample}.jpg", target_dir/f"test/{dir.name}_{sample}.jpg")

In [ ]:
resize_images("../data/original", "../data/resized", 10)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def generate_images(source_path, target_path, generations):
  
  datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    channel_shift_range = 10,
    fill_mode= 'nearest'
  )
  source_path = Path(source_path)
  target_path = Path(target_path)
    
  if target_path.exists():
    print("Target already exists, please delete it.")
    return 0
  
  target_path.mkdir(parents=True)
  for image in source_path.iterdir():
        
    image_name, image_ext = image.name.split(".")
    img = load_img(image)
    img = img_to_array(img)
    img = img.reshape((1,) + img.shape)
    
    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir=target_path, save_prefix=image_name, save_format="jpg"):
      i = i+1
      if i >= generations:
        break

In [ ]:
generate_images("../data/resized/train", "../data/generated/train", 100)
generate_images("../data/resized/test", "../data/generated/test", 100)